## AI Agent Core

**Goal:** Build intelligent agents that generate, plan, and optimize content using foundation models (like GPT, open-source LLMs, etc.) through the **LangChain framework.**

Build **modular, agentic components** that:
| Step | Agent               | Output                                             |
| ---- | ------------------- | -------------------------------------------------- |
| 4.1  | Topic Generator     | List of topics (title + description)               |
| 4.2  | Brief Writer        | Structured content briefs                          |
| 4.3  | Content Drafting    | First draft articles                               |
| 4.4  | Content Enhancement | Polished, SEO-optimized, publication-ready content |

agents are **composable** — they can call tools, access memory, use documents, or interact with APIs.

#### Concepts

| Concept            | Explanation                                                                            |
| ------------------ | -------------------------------------------------------------------------------------- |
| **LangChain**      | Python framework to build LLM-powered agents that can use tools, memory, and workflows |
| **PromptTemplate** | Structure + variables used to generate consistent outputs from LLMs                    |
| **Tools**          | Functions that your AI agent can call (e.g., search, summarize, recommend)             |
| **Chains**         | Sequences of steps the LLM performs (e.g., extract → summarize → plan)                 |
| **Memory**         | Lets agents remember past interactions                                                 |
| **Agents**         | Smart, autonomous systems that use all the above to complete tasks                     |


## Step 4.1 - AI Agent Core: Topic Generator

In [1]:
import os
import sys
import pandas as pd

In [2]:
# Add project root to sys path
"""
This is needed because this notebook file is under root/notebooks folder. 
Where as the agent files are under root/src/agents folder.
This is to navigate from notebooks to agents folder
"""

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
#print(project_root)
#print(sys.path)

if project_root not in sys.path:
    sys.path.append(project_root)

In [9]:
# Now we can import from src
from src.agents.topic_generator import generate_topics
from src.agents.brief_writer import generate_brief
from src.agents.content_drafter import generate_draft
from src.agents.content_polisher import polish_draft

In [4]:
# Load combined data
data_path = os.path.join(project_root, "data", "combined_data.csv")
#print(data_path)

if not os.path.exists(data_path):
    raise FileNotFoundError(f"Data file not found at: {data_path}")

combined_df = pd.read_csv(data_path, parse_dates=["publishedAt"])
print(f"Loaded data from: {data_path}")
print(f"Total articles/ posts: {len(combined_df)}")

combined_df.head()

Loaded data from: /Users/vinothhaldorai/Documents/Vinoth/PROJECTS/content-marketing-agent/data/combined_data.csv
Total articles/ posts: 46


,title,publishedAt,source,publishedDate
0,AI in content marketing: How creators and mark...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
1,AI in Marketing recent news | Content Marketin...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
2,A Complete Guide to Adopting AI in Content Mar...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
3,AI tools for content marketing : r/marketing,2025-07-08 23:50:58+00:00,Google Search,2025-07-08
4,Artificial Intelligence And The Future Of Cont...,2025-07-08 23:50:58+00:00,Google Search,2025-07-08


#### Generate topics using Agent

In [6]:
# Generate topics using agent
# We will pass recent titles from multiple platforms to LLM

titles = combined_df["title"].dropna().tolist()
# print(titles)

# Call the LLM agent
topics = generate_topics(titles)

# Display suggested topics
if isinstance(topics, list):
    print("Topic is returned as list")
    for i, topic in enumerate(topics, 1):
        print(f"Topic {i}: {topic['title']}")
        print(f"Description: {topic['description']}\n")
else:
    # fallback to raw response if JSON parsing failed
    print(topics)

# print(topics)

Topic is returned as list
Topic 1: AI-Powered Content Marketing: A Game Changer
Description: Exploring how creators and marketers leverage AI to enhance content creation and engagement.

Topic 2: Top AI Marketing Tools to Watch in 2025
Description: A roundup of the most innovative AI tools poised to transform digital marketing strategies in the near future.

Topic 3: Automating Your Marketing: The Future is AI
Description: An in-depth look at AI tools that enable complete automation in content marketing for efficiency and effectiveness.

Topic 4: Sustainable Food Marketing through AI Precision
Description: How AI is revolutionizing marketing strategies in the sustainable food sector for better targeting and results.

Topic 5: AI Content Generators: The Ultimate Guide
Description: A comprehensive guide to the best AI content generation tools that can streamline your writing process and boost productivity.



In [7]:
# Example usage
sample_keywords = ["AI in marketing", "SEO automation", "generative content"]
print(generate_topics(sample_keywords))

[{'title': 'The Future of Marketing: AI at the Helm', 'description': 'Exploring how AI technologies are reshaping marketing strategies and customer interactions.'}, {'title': 'Revolutionizing SEO with Automation', 'description': 'A look into how SEO automation tools can save time and improve ranking efficiency.'}, {'title': 'Generative Content: The Next Big Thing', 'description': 'Understanding how generative content is changing the landscape of digital marketing.'}, {'title': 'AI-Driven Insights for Better Marketing Decisions', 'description': 'How leveraging AI can provide actionable insights to enhance marketing campaigns.'}, {'title': 'Streamlining Content Creation with AI', 'description': 'An examination of how AI can automate and enhance the content creation process.'}]


## Step 4.2 - Brief Writer Agent

**Objective**

Automatically generate a structured content brief for each topic. This includes:
- Title suggestion
- Section-wise outline
- Tone and style
- Target audience
- Suggested call to action (CTA)

**Concepts**

| Concept                    | What You'll Learn                                           |
| -------------------------- | ----------------------------------------------------------- |
| **Prompt engineering**     | How to guide LLMs to produce structured, consistent outputs |
| **LangChain LLMChain**     | Reusing the same LangChain building block                   |
| **Prompt composition**     | Structuring multi-part outputs cleanly                      |
| **Separation of concerns** | Keeping agents modular, for reuse or replacement            |


In [8]:
# Test the brief_writer agent
sample_topic = "How AI is Revolutionizing Content Marketing"
brief = generate_brief(sample_topic, sample_topic)
print(brief)

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-NAva5sl7rvMGp1VcUWWHbtk6 on tokens per min (TPM): Limit 100000, Used 100000, Requested 138. Please try again in 59m36.96s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

In [9]:
topics = generate_topics(titles)  # list of dicts
print(topics)

for topic in topics:
    brief = generate_brief(title=topic["title"], description=topic["description"])
    print(brief)

[{'title': 'AI-Powered Content Marketing Strategies', 'description': 'Exploring how creators and marketers leverage AI tools to enhance engagement and drive conversions.'}, {'title': 'The Future of AI in Digital Marketing', 'description': 'A look at emerging AI technologies and their potential impact on content marketing strategies through 2025.'}, {'title': 'Top AI Marketing Tools You Need to Know', 'description': 'A roundup of the best AI marketing tools that can automate and optimize your marketing efforts in 2024.'}, {'title': 'Maximizing ROI with AI Content Generation', 'description': 'Ways to effectively use AI content generators to create high-quality content that boosts your marketing ROI.'}, {'title': 'AI and Sustainable Marketing Practices', 'description': 'How AI is bringing precision to sustainable food marketing and helping brands meet eco-friendly goals.'}]
{'title': 'Unlocking Engagement: AI-Powered Content Marketing Strategies That Convert', 'outline': ['Section 1: Intr

## Step 4.3 - Content Drafting Agent

**Goal**

Turn a topic + brief into a coherent, SEO-friendly first draft blog post.

**Concepts**

We You already generated a brief containing:
- Title
- Outline
- Tone
- Audience
- Call to Action

Now we pass the brief into an LLM to generate a full article draft.

In [10]:
# Test the content drafting agent
brief = {
    "title": "Harnessing AI in Marketing",
    "outline": "1. Introduction\n2. Key AI tools\n3. Personalization\n4. Case Studies\n5. Challenges\n6. Conclusion",
    "tone": "Professional yet approachable",
    "audience": "Digital marketers and content strategists",
    "cta": "Explore our AI content toolkit"
}

draft = generate_draft(brief)
print(draft)

# Harnessing AI in Marketing

## Introduction

In the ever-evolving landscape of digital marketing, artificial intelligence (AI) has emerged as a game-changer for businesses looking to connect with their audiences more effectively and efficiently. From automating mundane tasks to delivering personalized experiences, AI is reshaping how marketers strategize and implement their campaigns. As content strategists and digital marketers, understanding the potential of AI can not only enhance your marketing efforts but also provide a competitive edge in a crowded marketplace.

## Key AI Tools

The arsenal of AI tools available today is vast, and selecting the right ones can significantly impact your marketing strategy. Here are some key AI tools that are making waves:

1. **Chatbots**: Tools like Drift and Intercom provide real-time customer support and engage users through conversational interfaces. These chatbots can answer common queries, schedule appointments, and even guide users through

In [12]:
# Generate topics (just get one for testing)
topics = generate_topics(titles[:5])  # Pass a smaller list if needed

# Use only the first topic
topic = topics[0]
title = topic.get("title", "")
description = topic.get("description", "")

print(f"Title: {title}")
print(f"Description: {description}")

# Generate brief for the first topic
brief = generate_brief(title, description)
print("\n**Brief**")
print(brief)

# Generate draft for the first brief
draft = generate_draft(brief)
print("\n**Draft**")
print(draft)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-NAva5sl7rvMGp1VcUWWHbtk6 on tokens per min (TPM): Limit 100000, Used 100000, Requested 216. Please try again in 1h33m18.72s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}

## Step 4.4 - Content Polisher Agent

The Content Polisher is an optional but valuable refinement step. It:
- Improves flow, tone, and coherence
- Aligns content to a consistent voice
- Removes verbosity or filler content
- Adds optional enhancements like quotes, stats, or examples
- Adapts tone for audience (professional, witty, etc.)

In [ ]:

# Assume you already have a generated brief and draft
tone = brief["tone"]
audience = brief["audience"]
draft = generate_draft(brief)

polished_content = polish_draft(draft, tone, audience)
print(polished_content)
